# Neural Network Exercises

In these exercises, you will be building your own artificial neural network and seeing how adding different types of layers can affect the validation/testing accuracy. This is based off of the simple neural network with Keras tutorial, so you can reference that for further explanations as well.

In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

2024-07-17 18:26:20.810558: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-17 18:26:20.994799: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-07-17 18:26:20.994829: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-07-17 18:26:21.042443: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-17 18:26:22.081976: W tensorflow/stream_executor/platform/de

In [2]:
os.system('wget https://raw.githubusercontent.com/MedlyticsUniversal/Data/main/Week2/spoken_digit_manual_features.csv')

--2024-07-17 18:26:23--  https://raw.githubusercontent.com/MedlyticsUniversal/Data/main/Week2/spoken_digit_manual_features.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220478 (215K) [text/plain]
Saving to: ‘spoken_digit_manual_features.csv.32’

     0K .......... .......... .......... .......... .......... 23% 21.2M 0s
    50K .......... .......... .......... .......... .......... 46% 26.1M 0s
   100K .......... .......... .......... .......... .......... 69% 28.2M 0s
   150K .......... .......... .......... .......... .......... 92% 27.8M 0s
   200K .......... .....                                      100% 5.93M=0.01s

2024-07-17 18:26:23 (20.6 MB/s) - ‘spoken_digit_manual_features.csv.32’ saved [220478/220478]



0

## Load Training Data and Pre-processed Features

Your goal is to build a neural network that learns to classify which of the 5 speakers is recorded in a signal sample. Your prediction will be based off of features we've already pre-extracted for you and put into this CSV file: spectral centroid `SC`, spectral flatness `SF`, and maximum frequency `MF`.

In [3]:
# Load csv containing raw data, labels, and pre-processed features
spoken_df = pd.read_csv('spoken_digit_manual_features.csv', index_col = 0)
print(spoken_df.head(10))
print('\n')

# Set speakers
speakers = set(spoken_df['speaker'])
print(f'There are {len(speakers)} unique speakers in the dataset')

                file  digit   speaker  trial           SC        SF  \
0   5_yweweler_8.wav      5  yweweler      8  1029.497959  0.397336   
1    3_george_49.wav      3    george      4  1881.296834  0.387050   
2  9_yweweler_44.wav      9  yweweler      4  1093.951856  0.394981   
3  8_yweweler_33.wav      8  yweweler      3  1409.543285  0.487496   
4      7_theo_34.wav      7      theo      3   887.361601  0.396825   
5   1_jackson_45.wav      1   jackson      4  1007.568129  0.324100   
6  6_yweweler_18.wav      6  yweweler      1  1286.701352  0.498813   
7    9_george_35.wav      9    george      3  1405.092061  0.353083   
8   9_jackson_32.wav      9   jackson      3  1172.899961  0.477907   
9    8_george_26.wav      8    george      2  1959.977577  0.462901   

           MF  
0  745.878340  
1  323.943662  
2  244.648318  
3  392.350401  
4  130.640309  
5  216.306156  
6  400.715564  
7  447.239693  
8  114.892780  
9  320.537966  


There are 5 unique speakers in the datas

In [4]:
# Make dictionary to convert from speaker names to indices
name2int_dict = {name: ind for (ind, name) in enumerate(set(spoken_df['speaker']))}

y_labels = spoken_df['speaker']
# Set y_labels to be indices of speaker
y_labels = [name2int_dict[name] for name in y_labels]

Split data into train, validation, and test sets and standardize:

In [5]:
# Downselect to only the 3 columns of the dataset we are learning from, aka the features
X_data = spoken_df[['SC', 'SF', 'MF']].to_numpy()

# Decide how large to make validation and test sets
n_val = 250
n_test = 250

# Shuffle data before partitioning
X_data, y_labels = shuffle(X_data, y_labels, random_state = 25)

# Partition
X_data_test, y_labels_test = X_data[:n_test,:], y_labels[:n_test]
X_data_val, y_labels_val = X_data[n_test:n_test+n_val,:], y_labels[n_test:n_test+n_val]
X_data_train, y_labels_train = X_data[n_test+n_val:,:], y_labels[n_test+n_val:]

# Scale data
scaler = StandardScaler()
X_data_train=scaler.fit_transform(X_data_train)
X_data_val = scaler.transform(X_data_val)
X_data_test = scaler.transform(X_data_test)

Converting labels to "one-hot" vectors:

In [6]:
# Convert labels to onehot
y_labels_train = tf.keras.utils.to_categorical(y_labels_train, 5)
y_labels_val =  tf.keras.utils.to_categorical(y_labels_val, 5)
y_labels_test =  tf.keras.utils.to_categorical(y_labels_test, 5)

training_set = tf.data.Dataset.from_tensor_slices((X_data_train, y_labels_train))

2024-07-17 18:26:23.767297: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-07-17 18:26:23.767359: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-07-17 18:26:23.767384: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-af4e382a-c6a1-46d5-a014-268d0d4a7f76): /proc/driver/nvidia/version does not exist
2024-07-17 18:26:23.767790: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Additional Layers

Before you get to building your own neural network, we'll show you some examples of additional layers you can potetially add that we didn't go over in the tutorial. After reading over our explanations/example code and going through documentation, you'll be testing some of these out by putting together a neural network yourself.

### Dropout Layers

Dropout layers randomly omit, or drop, some elements of the output vector from the layer, which helps prevent overfitting and can improve the generalization of your neural network. The dropout rate can be any number between 0 and 1.

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout

```
# Example
d_r = 0.6
tf.keras.layers.Dropout(rate=d_r)
```

### Pooling Layers

A pooling layer reduces dimensionality (the size of each feature map) and "compresses" information by combining several output elements. Two common functions used for pooling are:
- **Average pooling**: calculating the average value for each patch on the feature map
- **Max pooling**: calculating the maximum value for each patch of the feature map

*Note: we won't apply pooling in this exercise notebook, but you'll be seeing more of it in future ones*

https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool1D

```
# Example
tf.keras.layers.MaxPool1D(pool_size=1)
```

### Activation Layers/Functions

An activation function looks at each "neuron" in your neural network and determines whether it should be activated (fired) or not, based on the relevancy of the neuron's input to the model's predictions. Some different activation functions you could look at are:
- **Softmax**: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Softmax
- **Sigmoid**: https://www.tensorflow.org/api_docs/python/tf/keras/activations/sigmoid
- **Softplus**: https://www.tensorflow.org/api_docs/python/tf/keras/activations/softplus
- **ReLU**: https://www.tensorflow.org/api_docs/python/tf/keras/layers/ReLU

```
# Example
tf.keras.layers.Softmax()
```

### Optimization Functions

- **Adam**: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam
    - Adam is computationally efficient, has little memory requirement, and is well suited for problems that are large in terms of data/parameter.
- **Adagrad**: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adagrad
    - Adagrad is an optimizer that is best used for sparse data. Some of its benefits are that it converges more quickly and doesn't need manual adjustment of the hyperparameter "learning rate".
- **SGD**: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/SGD
    - SGD is a **stochastic gradient descent** and momentum optimizer. SGD essentially helps gradient vectors move down loss functions towards the minimum point, leading to faster "converging."
- **RMSprop**: https://keras.io/api/optimizers/rmsprop/
    - As you may already know, the learning rate regulates how much the model can change based on the estimated error (which occurs every time the model's weights are updated). Instead of treating the learning rate as a hyperparamter, RMSprop is an optimization technique that relies on a changing, adaptive learning rate.

```
# Example code
l_r = .001 
tf.keras.optimizers.SGD(learning_rate=l_r)
```

## Putting Together Your Neural Network

Now you will experiment with adding different layers to your neural network. We've added some guiding comments to give you a place to start and test out, but we also strongly encourage you to go through all the documentation and use the Internet as well!

In [7]:
# Once you've gone through all the tests, play around with these rates to see if you can increase your accuracy.
l_r = .01
d_r = 0.5

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(8, input_shape=(3,), activation = tf.nn.relu))

### Test 1

In [8]:
# Run this cell as it is
#model.add(tf.keras.layers.Dense(8, activation = tf.nn.relu))
#model.add(tf.keras.layers.Dense(8, activation = tf.nn.relu))

# Output dimension needs to be number of classes in order for each to get a score
#model.add(tf.keras.layers.Dense(5))

# NOW SKIP down to the section that compiles and trains your model and run those cells.
# Check the pseudo-test accuracy and see how well the bare minimum performed.



### Test 2

In [9]:
# ADD activation layer here
#model.add(tf.keras.layers.Dense(8, activation=tf.nn.sigmoid))
# ADD activation layer here
#model.add(tf.keras.layers.Dense(8, activation = tf.nn.sigmoid))
# ADD activation layer here
#model.add(tf.keras.layers.Dense(8, activation = tf.nn.sigmoid))

# Output dimension needs to be number of classes in order for each to get a score
#model.add(tf.keras.layers.Dense(5))

# NOW SKIP down to the section that compiles and trains your model and re-run those cells.
# What do you notice about the testing/validation accuracy after test 2 in comparison to test 1?

### Test 3

In [10]:
# ADD activation layer here
model.add(tf.keras.layers.Dense(16, activation = tf.nn.relu))
# ADD activation layer here
model.add(tf.keras.layers.Dense(16, activation = tf.nn.relu))
# ADD activation layer here 
model.add(tf.keras.layers.Dense(16, activation = tf.nn.relu))

model.add(tf.keras.layers.Dense(24, activation = tf.nn.relu))

# Output dimension needs to be number of classes in order for each to get a score
model.add(tf.keras.layers.Dense(5, tf.nn.softmax))

# ADD dropout layer here
tf.keras.layers.Dropout(rate=d_r)
# Now skip down to the section that compiles and trains your model and re-run those cells.
# What do you notice about the testing/validation accuracy after test 3 in comparison to tests 1 & 2?

### Test 4

Now go back down to the cell where you compiled your model and this time, change the optimizer. It's been set to `Adam` as default but as we showed you above, there are other functions that you can test out.

## Compiling and Training Your Model

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 32        
                                                                 
 dense_1 (Dense)             (None, 16)                144       
                                                                 
 dense_2 (Dense)             (None, 16)                272       
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                                 
 dense_4 (Dense)             (None, 24)                408       
                                                                 
 dense_5 (Dense)             (None, 5)                 125       
                                                                 
Total params: 1,253
Trainable params: 1,253
Non-trainabl

In [12]:
model.compile(loss = tf.keras.losses.categorical_crossentropy, 
              optimizer = tf.keras.optimizers.Adam(learning_rate=l_r),
              metrics = ['accuracy'])   

Specify number of epochs and batch size, and fit the model to data:

In [13]:
EPOCHS = 50
batch_size = 100

training_set = training_set.batch(batch_size) # Set batch size

def run_model(model, EPOCHS, training_set):
    for epoch in range(EPOCHS):
        for signals, labels in training_set:
            tr_loss, tr_accuracy = model.train_on_batch(signals, labels)
        val_loss, val_accuracy = model.evaluate(X_data_val, y_labels_val)
        print(('Epoch #%d\t Training Loss: %.2f\tTraining Accuracy: %.2f\t'
            'Validation Loss: %.2f\tValidation Accuracy: %.2f')
            % (epoch + 1, tr_loss, tr_accuracy,
            val_loss, val_accuracy))

run_model(model, EPOCHS, training_set)

8/8 [==============================] - 0s 2ms/step - loss: 1.4240 - accuracy: 0.4400
Epoch #1	 Training Loss: 1.44	Training Accuracy: 0.43	Validation Loss: 1.42	Validation Accuracy: 0.44
8/8 [==============================] - 0s 2ms/step - loss: 1.2527 - accuracy: 0.5360
Epoch #2	 Training Loss: 1.22	Training Accuracy: 0.51	Validation Loss: 1.25	Validation Accuracy: 0.54
8/8 [==============================] - 0s 2ms/step - loss: 1.1878 - accuracy: 0.5120
Epoch #3	 Training Loss: 1.11	Training Accuracy: 0.52	Validation Loss: 1.19	Validation Accuracy: 0.51
8/8 [==============================] - 0s 2ms/step - loss: 1.1736 - accuracy: 0.5480
Epoch #4	 Training Loss: 1.18	Training Accuracy: 0.49	Validation Loss: 1.17	Validation Accuracy: 0.55
8/8 [==============================] - 0s 2ms/step - loss: 1.1460 - accuracy: 0.5280
Epoch #5	 Training Loss: 1.10	Training Accuracy: 0.52	Validation Loss: 1.15	Validation Accuracy: 0.53
8/8 [==============================] - 0s 3ms/step - loss: 1.1217

In [14]:
# Check performance on test set
test_loss, test_accuracy = model.evaluate(X_data_test, y_labels_test)

8/8 [==============================] - 0s 2ms/step - loss: 0.9584 - accuracy: 0.6160


Now create your own model and/or modify the existing model, and try to find the highest, appropriate testing and validation accuracies!

In [21]:
l_r = .01
d_r = 0.6

model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Dense(8, input_shape=(3,)))
model2.add(tf.keras.layers.Dense(10))
model2.add(tf.keras.layers.Dense(10, tf.nn.relu))
model2.add(tf.keras.layers.Dropout(rate=d_r))
model2.add(tf.keras.layers.Dense(5))
model2.compile(loss = tf.keras.losses.categorical_crossentropy, 
              optimizer = tf.keras.optimizers.Adam(learning_rate=l_r),
              metrics = ['accuracy'])   

run_model(model2, 50, training_set)

2024-07-17 18:34:16.903554: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2024-07-17 18:34:16.951559: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2024-07-17 18:34:16.999583: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
8/8 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.1480
Epoch #1	 Training Loss: nan	Training Accuracy: 0.19	Validation Loss: nan	Validation Accuracy: 0.15
8/8 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.1480
Epoch #2	 Training Loss: nan	Training Accuracy: 0.19	Validation Loss: nan	Validation Accuracy: 0.15
8/8 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.1480
Epoch #3	 Training Loss: nan	Training Accuracy: 0.19	Validation Loss: nan	Validation Accuracy: 0.15
2024-07-17 18:34:18.244371: W tensorflow/core/data/root

In [20]:
# Once you've gone through all the tests, play around with these rates to see if you can increase your accuracy.
l_r = .01
d_r = 0.5

model3 = tf.keras.Sequential()
model3.add(tf.keras.layers.Dense(8, input_shape=(3,), activation = tf.nn.relu))

model3.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
model3.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
model3.add(tf.keras.layers.Dropout(rate=.2))

model3.add(tf.keras.layers.Dense(5, tf.nn.softmax))
model3.compile(loss = tf.keras.losses.categorical_crossentropy, 
              optimizer = tf.keras.optimizers.Adam(learning_rate=l_r),
              metrics = ['accuracy'])   

run_model(model3, 40, training_set)
test_loss, test_accuracy = model3.evaluate(X_data_test, y_labels_test)

2024-07-17 18:33:25.555632: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
8/8 [==============================] - 0s 3ms/step - loss: 1.1492 - accuracy: 0.5640
Epoch #1	 Training Loss: 1.17	Training Accuracy: 0.52	Validation Loss: 1.15	Validation Accuracy: 0.56
2024-07-17 18:33:26.287574: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2024-07-17 18:33:26.379593: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
8/8 [==============================] - 0s 5ms/step - loss: 1.0987 - accuracy: 0.5640
Epoch #2	 Training Loss: 1.08	Training Accuracy: 0.54	Validation Loss: 1.10	Validation Accuracy: 0.56
2024-07-17 18:33:26.671571: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2024-07-17 18:33:26.872564: W tensorflow/core/data/root_dataset.cc:266] Optimization loop fail

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=af4e382a-c6a1-46d5-a014-268d0d4a7f76' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>